# Utilities

In [ ]:
import os
from PIL import Image
from pathlib import Path
import glob
import subprocess
import shutil

current_path = Path('.').resolve()
print(str(current_path))

## Script to convert BMP kodak dataset images to PNG image for later comparison

In [ ]:
source_path = str(current_path)+'/IMAGES/BMP_IMAGES'
print("Source: ",source_path)
destination_path = str(current_path)+'/IMAGES/PNG_IMAGES'
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

In [ ]:
for file in glob.glob(str(source_path)+'/*.bmp'):
    image = Image.open(file)
    image = image.convert('RGB')
    print(Path(file).stem)
    image.save(str(destination_path)+'/'+Path(file).stem+'.png','png')
    image.close()

## Script to convert PNG images in YUV lossless for VVC encoding
In order to perform the conversion a working installation of FFmpeg is needed


In [ ]:
source_path = str(current_path)+'/IMAGES/PNG_IMAGES'
print("Source: ",source_path)
destination_path = str(current_path)+'/IMAGES/YUV_IMAGES'
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

In [ ]:
for file in os.listdir(source_path):
    image = Image.open(source_path+'/'+file)
    with open(str(destination_path)+'/'+str(Path(file).stem)+'.txt', 'w') as f:
        f.write(str(image.width)+'x'+str(image.height))
    source_file = str(source_path) + '/' + str(file)
    destination_file = str(destination_path) + '/' + str(Path(file).stem) + '.yuv'
    subprocess.run(['ffmpeg','-y','-i',str(source_file),'-f','rawvideo','-sws_flags','accurate_rnd+full_chroma_int+full_chroma_inp','-vf','scale=in_range=full:out_range=full,format=yuv420p10le',str(destination_file)],stdout = subprocess.DEVNULL)
    print(Path(file).stem)

## Script to convert YUV images back into PNG

In [ ]:
dimensions_path = str(current_path)+'/IMAGES/YUV_IMAGES'
print('Dimensions: ',dimensions_path)
quantizer_parameters = [42,36,33,31,30]

In [ ]:
for qi in quantizer_parameters:
    source_path = str(current_path)+'/IMAGES/VVC/'+str(qi)+'/YUV'
    print("Source: ",source_path)
    destination_path = str(current_path)+'/IMAGES/VVC/'+str(qi)
    if os.path.isdir(destination_path) & os.path.exists(destination_path):
        print("Destination: ",destination_path)
    else:
        os.mkdir(destination_path)

    for file in os.listdir(source_path):
        if file.lower().endswith('.yuv'):
            dimensions = open(str(dimensions_path)+'/'+str(Path(file).stem)+'.txt').readline()
            source_file = str(source_path) + '/' + str(file)
            destination_file = str(destination_path) + '/' + str(Path(file).stem) + '.png'
            subprocess.run(['ffmpeg','-y','-sws_flags','accurate_rnd+full_chroma_int+full_chroma_inp','-s',str(dimensions),'-pix_fmt','yuv420p10le','-i',str(source_file),'-f','image2','-vframes','1','-pix_fmt','rgb24',str(destination_file)],stdout = subprocess.DEVNULL)
            print(Path(file).stem)


## Extract images for downsampling and compression in order to use them in the PDF document

In [ ]:
datasets = {
    'PNG':'/IMAGES/PNG_IMAGES',
    'JPEG_1':'/IMAGES/JPEG/2',
    'JPEG_2':'/IMAGES/JPEG/6',
    'JPEG_3':'/IMAGES/JPEG/19',
    'JPEG_4':'/IMAGES/JPEG/23',
    'JPEG_5':'/IMAGES/JPEG/30',
    'JPEG2000_1':'/IMAGES/JPEG2000/175',
    'JPEG2000_2':'/IMAGES/JPEG2000/140',
    'JPEG2000_3':'/IMAGES/JPEG2000/115',
    'JPEG2000_4':'/IMAGES/JPEG2000/72',
    'JPEG2000_5':'/IMAGES/JPEG2000/56',
    'BPG_1':'/IMAGES/BPG/49',
    'BPG_2':'/IMAGES/BPG/44',
    'BPG_3':'/IMAGES/BPG/41',
    'BPG_4':'/IMAGES/BPG/38',
    'BPG_5':'/IMAGES/BPG/36',
    'VVC_1':'/IMAGES/VVC/42',
    'VVC_2':'/IMAGES/VVC/36',
    'VVC_3':'/IMAGES/VVC/33',
    'VVC_4':'/IMAGES/VVC/31',
    'VVC_5':'/IMAGES/VVC/30',
    'mbt2018_1':'/IMAGES/NET/mbt2018_1',
    'mbt2018_2':'/IMAGES/NET/mbt2018_2',
    'mbt2018_3':'/IMAGES/NET/mbt2018_3',
    'mbt2018_4':'/IMAGES/NET/mbt2018_4',
    'mbt2018_5':'/IMAGES/NET/mbt2018_5',
    'mbt2018_mean_1':'/IMAGES/NET/mbt2018_mean_1',
    'mbt2018_mean_2':'/IMAGES/NET/mbt2018_mean_2',
    'mbt2018_mean_3':'/IMAGES/NET/mbt2018_mean_3',
    'mbt2018_mean_4':'/IMAGES/NET/mbt2018_mean_4',
    'mbt2018_mean_5':'/IMAGES/NET/mbt2018_mean_5',
    'cheng2020_attn_1':'/IMAGES/NET/cheng2020_attn_1',
    'cheng2020_attn_2':'/IMAGES/NET/cheng2020_attn_2',
    'cheng2020_attn_3':'/IMAGES/NET/cheng2020_attn_3',
    'cheng2020_attn_4':'/IMAGES/NET/cheng2020_attn_4',
    'cheng2020_attn_5':'/IMAGES/NET/cheng2020_attn_5',
    'cheng2020-anchor_1':'/IMAGES/NET/cheng2020-anchor_1',
    'cheng2020-anchor_2':'/IMAGES/NET/cheng2020-anchor_2',
    'cheng2020-anchor_3':'/IMAGES/NET/cheng2020-anchor_3',
    'cheng2020-anchor_4':'/IMAGES/NET/cheng2020-anchor_4',
    'cheng2020-anchor_5':'/IMAGES/NET/cheng2020-anchor_5'
}

In [ ]:
destination_path = str(current_path)+'/IMAGES/TO_PDF'
print("Destination: ",destination_path)
if os.path.isdir(destination_path) & os.path.exists(destination_path):
    try:
        files = os.listdir(destination_path)
        for file in files:
            file_path = os.path.join(destination_path, file)
            if os.path.isfile(file_path):
                os.remove(file_path)
        print("Destination directory has been emptied")
    except OSError:
        print("Error occurred while deleting files in the destination directory")
else:
    os.mkdir(destination_path)

In [ ]:
filename_to_copy = 'IMG0004.png'
for name, path in datasets.items():
    source_path = str(current_path)+path
    shutil.copy(str(source_path+'/'+filename_to_copy),str(destination_path+'/'+name+'_'+filename_to_copy))
    print('Copied '+filename_to_copy+' from '+source_path)